In [13]:
import numpy as np
import cv2 as cv
import matech_utilities as mu

# Parameters for lucas kanade optical flow
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

In [31]:
cap = cv.VideoCapture(0)
ret, frame = cap.read()
vh, vw, _ = frame.shape
r, h, c, w = np.int32((vh/4, vh/2, vw/4, vw/2))
org_box = np.array([[c, r, 1],
                    [c, r + h, 1],
                    [c + w, r + h, 1],
                    [c + w, r, 1]
                    ])
_, prev_box_center = mu.draw_box_homogeneous(box, frame, True)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    res, center = mu.draw_box_homogeneous(org_box, frame, True)

    # Display the resulting frame
    cv.imshow('frame',res)
    if cv.waitKey(1) & 0xFF == ord('c'):
        break

# c. mask
org_mask = mu.create_mask(frame)
org_mask[r:r + h, c:c + w] = 255
# c. features
old_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
_, p00, _ = mu.extract_features(old_gray, resize=False, mask=org_mask)
p0 = np.array([p.pt for p in p00], dtype=np.float32).reshape((-1, 1, 2))
prev_point_center = np.mean(p0.reshape((-1, 2)), axis=0)


prev_box = org_box

while True:
    ret, frame = cap.read()
    
    # Optic flow
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    p1, st, err = cv.calcOpticalFlowPyrLK(
        old_gray, frame_gray, p0, None, **lk_params)
    # pair points
    condition = np.array(st == 1)  
    good_new = p1[condition]
    good_old = p0[condition]
    point_center = np.mean(good_new, axis=0)
    # homography
    h, m = cv.findHomography(good_old, good_new, cv.RANSAC, 5.0)
    box = mu.transform_with_homography(h, prev_box)
    box[:, :-1] = box[:, :-1] - ((box_center-prev_box_center) - (point_center-prev_point_center))
        
    for a, b in good_new:
        frame = cv.circle(frame, (a, b), 5, (0, 255, 0), -1)
    res, box_center = mu.draw_box_homogeneous(box, frame, True)
    print((box_center-prev_box_center) - (point_center-prev_point_center))
    cv.imshow('frame',frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
    # update
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
    prev_box = box
    prev_box_center = box_center
    prev_point_center = point_center
    np.mean(good_new, axis=0)

# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

[  8.30172729  75.4770813 ]
[-12.91461182   4.31237793]
[-4.14477539  2.83340454]
[-4.49258423  3.86746216]
[-4.95523071  3.63226318]
[-4.57659912  2.47827148]
[-3.29159546  1.49359131]
[-3.69308472 -0.49801636]
[-2.61123657 -1.50131226]
[-1.25332642 -3.03329468]
[-0.3621521  -2.53543091]
[-0.08007812 -1.51489258]
[-0.64376831 -2.20907593]
[-0.30126953 -2.44992065]
[-1.55712891 -1.20822144]
[-2.55374146 -1.33834839]
[-2.20028687 -3.59954834]
[-3.12225342 -4.70840454]
[-2.07250977 -4.62069702]
[-1.99240112 -4.89819336]
[-1.73095703 -4.64437866]
[-1.66912842 -4.7855835 ]
[-0.88150024 -4.53939819]
[-1.05834961 -4.41781616]
[ 0.62127686 -4.1859436 ]
[ 0.93951416 -4.30444336]
[ 0.54937744 -4.09082031]
[ 1.08135986 -3.75921631]
[ 1.96786499 -4.25830078]
[ 2.23776245 -3.36566162]
[ 1.71343994 -3.03451538]
[ 1.45285034 -2.88818359]
[ 1.80187988 -2.98403931]
[ 2.05404663 -2.93347168]
[ 0.52978516 -3.15202332]
[ 0.73672485 -1.91921997]
[-0.04605103 -2.01417542]
[-0.10476685 -1.11541748]
[-2.1961

[-34.41455078 -38.02172852]
[-85.18823242 -93.22903442]
[-144.17285156 -132.17385864]
[ 208.81997681  755.90794373]
[  379.3838501  -1665.66667175]
[ -231.16668701  1520.42578125]
[ 88.06494141  21.36811829]
[-507.30563354 -736.02072144]
[ 149.61651611  200.8293457 ]
[-223.39059448 -355.48649597]
[  52.7199707   412.09008789]
[ 614.43261719  116.38928223]
[ 27.48031616 -43.97412109]
[-30.18212891  11.46607971]
[-1.60797119  5.28520203]
[-0.37017822  2.15869141]
[ 7.44848633 -4.18823242]
[-15.15890503   9.57141113]
[ 18.11474609 -16.93725586]
[-5.72702026  6.72406006]
[ 20.92251587 -31.37426758]
[ 1.60559082 -8.47573853]
[ 34.74026489 -49.42350769]
[ 10.6746521 -19.0055542]
[ -7.28787231  10.55607605]
[-5.69113159  9.88589478]
[ 1.33563232  3.26394653]
[ 26.69210815 -37.44158936]
[ -7.75030518  14.87460327]
[ 4.4005127  -1.31903076]
[-27.89877319  35.71948242]
[-1.1493988  -4.87695312]
[ -0.30717468 -11.56726074]
[-1.20262146 -7.63983154]
[-2.82287598 -0.95227051]
[ 1.25878906 -6.004638

AttributeError: 'NoneType' object has no attribute 'dot'

In [32]:
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

In [24]:
np.mean(good_new, axis=0)

array([ 457.42929077,  329.11203003], dtype=float32)

In [23]:
good_new

array([[ 446.81903076,  317.96652222],
       [ 437.41342163,  270.55874634],
       [ 488.05545044,  398.81079102]], dtype=float32)